# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


##### Gathering Miscellaneous url info

In [ ]:
# google (places api) nearby search reqiest url:   https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters
    #  Documentation:  https://developers.google.com/maps/documentation/places/web-service/search-nearby?hl=en_US

# google (geocoding api) https://maps.googleapis.com/maps/api/geocode/outputFormat?parameters
    #   Documentation:  https://developers.google.com/maps/documentation/geocoding/overview?hl=en_US


# google mapsJavaAPI   MapsJavaScriptAPI uses this script tag that I know nothing about.
    # <script async
    #     src="https://maps.googleapis.com/maps/api/js?key=YOUR_API_KEY&callback=initMap">
    # </script>
    # Documentation: https://developers.google.com/maps/documentation/javascript/overview?hl=en_US






### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
weather_data_csv = "weather_data.csv"
weather_data_df = pd.read_csv(weather_data_csv)
weather_data_df


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
gmaps.configure(api_key = g_key)

In [ ]:
# Store "Lat" and "Lng" into locations
locations = weather_data_df[['Lat', 'Lon']].astype(float)

# Store 'Humidity' into humidity 
humidity = weather_data_df['Humidity'].astype(float)

In [ ]:
# Create a humidity heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
good_humidity = weather_data_df[(weather_data_df['Humidity'] < 60)].dropna()
best_humidity = good_humidity[(good_humidity['Humidity'] > 55)].dropna()
good_temp = best_humidity[(best_humidity['Temp'] > 297)].dropna()
best_weather = good_temp[(good_temp['Temp'] < 302)].dropna()
best_weather

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
    }

# Iterate through 
for index, row in best_weather.iterrows():
    # get lat, lon from df
    lat = row["Lat"]
    lon = row["Lon"]
    
    params["location"] = f"{lat},{lon}"
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    address = response.json()
    try:
        best_weather['Hotel Name'] = address["results"][0]["name"]
    except:    
        print('skipping error')
        pass
best_weather

In [ ]:
humidity_level = best_weather["Humidity"].tolist()
humidity_level
hotel_name = best_weather['Hotel Name'].tolist()
hotel_name

In [ ]:
# Note: Do not change the code in this cell 

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in best_weather.iterrows()]
hotel_locations = best_weather[["Lat", "Lon"]].astype(float)
hotel_locations

In [ ]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(hotel_locations, weights= humidity_level,
                                dissipating=False, max_intensity=100,
                                point_radius=2)

fig.add_layer(heat_layer)

fig

In [ ]:
# Creat Bank Marker Map
hotel_layer = gmaps.symbol_layer(
hotel_locations, fill_color = 'rgba(100,150,0,0.4)',
    stroke_color= 'rgba(0,150,0,0.4)', scale=2,
    info_box_content=[f"Hotel Name:{hotel}" for hotel in hotel_name]
)

fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

In [ ]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig
